# ZAFER ULGUR - 2022325

## MSc in Applied Software Development
## Continuous Assessment - 1
## Stock Market

### Information Retrieval and Data Mining

In [ ]:
#<ZAFER ULGUR>
#yahooFinance
!pip install -q yfinance
#beautifulSoup
!pip install bs4
#html5lib
#!pip install html5lib
#linearRegression
!pip install statsmodels

In [ ]:
#import libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader.data import DataReader
import requests as req
from bs4 import BeautifulSoup
import plotly.graph_objects as g
from plotly.subplots import make_subplots as sp
import statsmodels.formula.api as smf

In [ ]:
#getStockItems
from datetime import datetime
#stockList = ['F', 'BA', 'GM', 'VOD']
#stockList = ['f', 'ba', 'gm', 'vod']
stockList = []

stList = []
stList.append(['F','ford'])
stList.append(['BA','boeing'])
stList.append(['GM','general-motors'])
stList.append(['VOD','vodafone'])

#print(*stockList)

endDate = datetime.now()
startDate = datetime(endDate.year - 5, endDate.month, endDate.day)
compList = []
for stockItem in stList:
    globals()[stockItem[0]] = yf.download(stockItem[0], startDate, endDate)
    compList.append(globals()[stockItem[0]])
    stockList.append(stockItem[0])
    print(stockItem[0])

In [ ]:
#addColumn for companyName
#compList = [F, BA, GM, VOD]

for comp, compName in zip(compList, stList):
    comp["Company"] = compName[1]
df = pd.concat(compList, axis=0)
df.tail(10)

In [ ]:
#getStockItemSummary
#F.describe()  
#BA.describe()  
#GM.describe()  
VOD.describe()  

In [ ]:
#getStockItemInfo
print("Ford Info")
print("--------------")
F.info() 
#print("--------------")
#print("Boeing Info")
#print("--------------")
#BA.info()
#print("--------------")
#print("G.Motor Info")
#print("--------------")
#GM.info()
#print("--------------")
#print("Vodafone Info")
#print("--------------")
#VOD.info()

In [ ]:
#HistoricalStockData for procedural paradigm
def getStockData(stockList):
    historicData = pd.DataFrame()
    for i in stockList:
        #extractStockData for stockName
        stockData = yf.Ticker(i[0])
        hist = stockData.history(period='max').Close
        historicData[i[0]] = hist 
    return historicData   

In [ ]:
historicalStockData = getStockData(stList)
historicalStockData.tail(5)

In [ ]:
# to csv
historicalStockData.to_csv('historic.csv')

In [ ]:
#read csv
dfHist = pd.read_csv('historic.csv')
dfHist.head()

In [ ]:
dfHist.info()

In [ ]:
dfHist.shape

In [ ]:
dfHist.describe(include="all")

In [ ]:
#findNullData
dfHist.isnull().sum()

In [ ]:
# replace NA values in GM with "0"
dfHist["GM"].fillna(0, inplace = True)
# replace NA values in VOD with "0"
dfHist["VOD"].fillna(0, inplace = True)

In [ ]:
def getData(sList):
    revList = []
    rev = pd.DataFrame(columns=["Date", "Revenue", "Company"])   
    for detail in sList:
        #crawling area
        url=f'https://www.macrotrends.net/stocks/charts/{detail[0]}/{detail[1]}/revenue'
        #Save the text of the response
        htmlText = req.get(url).text
        #Parse the html data using beautiful_soup.
        soup = BeautifulSoup(htmlText, "html.parser")
        tables = soup.find_all('table')
        tableIndex=0
        for index, table in enumerate(tables):
            if ('Annual Revenue'in str(table)):
                tableIndex=index
        for row in tables[tableIndex].tbody.find_all("tr"):
            col = row.find_all("td")
            #print(col)
            if (col!=[]):
                date =col[0].text
                # removeDollarComma
                revenue =col[1].text.replace("$", "").replace(",", "")
                company = detail[1] 
            #print([{'Date':date,'Revenue':revenue}])
            rev = pd.concat([rev, pd.DataFrame.from_records([{'Date':date,'Revenue':revenue, 'Company':company}])])
        #print(rev)
        # removeNullVal data
        rev = rev[rev['Revenue']!='']   
    revList.append(rev)
    return revList[0]


        #print(revList)

In [ ]:
#getRevenueData
revenue = getData(stList)

In [ ]:
# displaying List
revenue.tail(5)

In [ ]:
# closingPriceForLastFiveYear
r=1
sLen=len(compList)
if sLen%2==0:
    r=sLen/2
else:
    r=(sLen+1)/2

plt.figure(figsize=(15, 10))

#fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)

for i, company in enumerate(compList, 1):
    plt.subplot(int(r), int(r), i)
    company['Adj Close'].plot()
    plt.ylabel('Adjusted Close')
    plt.xlabel(None)
    plt.title(f"{stList[i - 1][1]}", fontsize=24)
    
plt.tight_layout()

In [ ]:
ma_day = [15, 30, 60]

for ma in ma_day:
    for comp in compList:
        colName = f"MA for {ma} days"
        comp[colName] = comp['Adj Close'].rolling(ma).mean()

In [ ]:
fig, axes = plt.subplots(nrows=int(r), ncols=int(r))
fig.set_figheight(10)
fig.set_figwidth(15)
a=0
b=0
c=0
for i in compList:
    i[['Adj Close', 'MA for 15 days', 'MA for 30 days', 'MA for 60 days']].plot(ax=axes[int(a),int(b)])
    axes[int(a),int(b)].set_title(i.Company.unique()[0])
    if a>b:
        c=b
        b=a
        a=c
    else:
        a+=1
fig.tight_layout()

In [ ]:
plt.figure(figsize=(12, 7))

for i, company in enumerate(compList, 1):
    plt.subplot(int(r), int(r), i)
    company['Daily Data'] = company['Adj Close'].pct_change()
    company['Daily Data'].hist(bins=50)
    plt.ylabel('Daily Data')
    plt.title(f'{stList[i - 1][1]}')
    
plt.tight_layout()

In [ ]:
#generic graph procedural method
def gr(grapList, gTitle, xTitle, yTitle):
    fig = sp(shared_xaxes=True, subplot_titles=(yTitle), vertical_spacing = .5)
    if 'Revenue' in grapList:
        for i in grapList.Company.unique():
            filterList = grapList[grapList.Company == i]
            fig.add_trace(g.Scatter(x=pd.to_datetime(filterList.Date, infer_datetime_format=True), y=filterList.Revenue.astype("float"), name = i))
    else:
        for i in grapList:
            if i != 'Date':
                fig.add_trace(g.Scatter(x=pd.to_datetime(grapList.Date, infer_datetime_format=True), y=grapList[i].astype("float"), name = i))
    fig.update_xaxes(title_text=xTitle)
    fig.update_yaxes(title_text=yTitle)
    fig.update_layout(showlegend=False, height=1000, title=gTitle, xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
gr(revenue, "Revenue","Date","Revenue ($ Millions)")

In [ ]:
gr(dfHist,'Price','Date',"Price ($)")

### Algorithms & Mathematical Methods

In [ ]:
# calcLog for the periodBased on AdjClose
F['F'] = np.log(F['Adj Close'] / F['Adj Close'].shift(1))
BA['BA'] = np.log(BA['Adj Close'] / BA['Adj Close'].shift(1))
GM['GM'] = np.log(GM['Adj Close'] / GM['Adj Close'].shift(1))
VOD['VOD'] = np.log(VOD['Adj Close'] / VOD['Adj Close'].shift(1))

In [ ]:
# X (F, BA, GM) - Y (VOD)
dfMath = DataReader(stockList, 'yahoo', startDate, endDate)['Adj Close']
#pd.concat([F['F'], BA['BA'],  GM['GM'], VOD['VOD']], axis = 1).dropna()

## toTheCsvFile
dfMath.to_csv("linear_regression.csv")

dfMath.head()

In [ ]:
plt.figure(figsize = (15, 6))
plt.rcParams.update({'font.size': 14})
plt.xlabel("F Returns")
plt.ylabel("VOD returns")
plt.title("Daily Data Last 5 Years")
plt.scatter(dfMath['F'], dfMath['VOD'])
plt.show()


In [ ]:
# Make a new item returns DataFrame
sItemReturns = dfMath.pct_change()
sItemReturns.head()



In [ ]:
# show a linear relationship
for i in stList:
    sns.jointplot(x=i[0], y=i[0], data=sItemReturns, kind='scatter', color='orangered')

In [ ]:
sListTmp = []
for stockItemOne in stList:
    for stockItemTwo in stList:
        if stockItemOne[0] != stockItemTwo[0] and stockItemTwo[0] not in sListTmp:
            sns.jointplot(x=stockItemOne[0], y=stockItemTwo[0], data=sItemReturns, kind='scatter', color='magenta')
    sListTmp.append(stockItemOne[0])

In [ ]:
sns.heatmap(sItemReturns.corr(), annot=True, cmap='autumn')

In [ ]:
sns.heatmap(dfMath.corr(), annot=True, cmap='winter')

In [ ]:
def getLinearRegression(stockItemOne, stockItemTwo):
    # 2a. Fit a simple linear regression model to the data using statsmodels 
    # Create an instance of the class OLS
    slrSmModel = smf.ols(f'{stockItemOne} ~ {stockItemTwo}', data=dfMath)

    # statsmodels calculates beta_0 and beta_1 here
    slrSmModelFirst = slrSmModel.fit()

    # Summary
    #print(slrSmModelFirst.summary()) 

    paramSlr = slrSmModelFirst.params
    print("\n------------------------------------------------------------------")
    print("INTERCEPT and SLOPE\n")
    #setMsgText
    interceptMsg = "The INTERCEPT in the statsmodels regression model =" 
    slopeMsg = "The SLOPE in the statsmodels regression model ="
    print(interceptMsg, np.round(paramSlr.Intercept, 4))
    print(slopeMsg, np.round(paramSlr[stockItemTwo], 4))

In [ ]:
sListTmp = []
for stockItemOne in stList:
    for stockItemTwo in stList:
        if stockItemOne[0] != stockItemTwo[0] and stockItemTwo[0] not in sListTmp:
            print(#"\n++++++++++++++++++++++++++++++++++++++++++++++++++",
                  "\n++++++++++++++++++++++++++++++++++++++++++++++++++\n",
                  stockItemOne[1], " - ", stockItemTwo[1], 
                  " Linear Regression : \n")
            getLinearRegression(stockItemOne[0],stockItemTwo[0])
    sListTmp.append(stockItemOne[0])

### Programming Paradigms
#### OOP Model

In [ ]:
class StockMarket:
    def __init__(s, stockItemList):
        s.endDate = datetime.now()
        s.startDate = datetime(endDate.year - 10, endDate.month, endDate.day)
        s.stockList = stockItemList
        cList = []
        for stockItem in s.stockList:
            globals()[stockItem[0]] = yf.download(stockItem[0], startDate, endDate)
            cList.append(globals()[stockItem[0]])
        s.compList = cList
        for comp, compName in zip(s.compList, s.stockList):
            comp["Company"] = compName[1]
            s.df = pd.concat(s.compList, axis=0)
    def getStockData(s):
        historicData = pd.DataFrame()
        for i in s.stockList:
            stockData = yf.Ticker(i[0])
            hist = stockData.history(period='max').Close
            historicData[i[0]] = hist 
        s.historicData = historicData
    def setToCsv(s):
        s.historicData.to_csv('historic_class.csv')
    def getReadAndCleanCsv(s):
        dfHistory = pd.read_csv('historic_class.csv')
        for stock in s.stockList:
            if 0 != dfHistory.isnull()[f'{stock[0]}'].sum():
                dfHistory[f'{stock[0]}'].fillna(0, inplace = True)
        s.data = dfHistory

In [ ]:
class stockMarketCrawling:
    def __init__(cr, stockItemList):
        cr.stockList = stockItemList
        cr.data = []
        yTitle = "Revenue ($ Millions)"
        gTitle = "Revenue"
        fig = sp(shared_xaxes=True, subplot_titles=("Stock Market"), vertical_spacing = .5)
        rev = pd.DataFrame(columns=["Date", "Revenue", "Company"])   
        for detail in cr.stockList:
            url=f'https://www.macrotrends.net/stocks/charts/{detail[0]}/{detail[1]}/revenue'
            htmlText = req.get(url).text
            soup = BeautifulSoup(htmlText, "html.parser")
            tables = soup.find_all('table')
            tableIndex=0
            for index, table in enumerate(tables):
                if ('Annual Revenue'in str(table)):
                    tableIndex=index
            for row in tables[tableIndex].tbody.find_all("tr"):
                col = row.find_all("td")
                if (col!=[]):
                    date =col[0].text
                    revenue =col[1].text.replace("$", "").replace(",", "")
                    company = detail[1] 
                rev = pd.concat([rev, pd.DataFrame.from_records([{'Date':date,'Revenue':revenue, 'Company':company}])])
            rev = rev[rev['Revenue']!='']   
        for i in rev.Company.unique():
            filterList = rev[rev.Company == i]
            fig.add_trace(g.Scatter(x=pd.to_datetime(filterList.Date, infer_datetime_format=True), y=filterList.Revenue.astype("float"), name = i))
        cr.data.append(rev)
        
        fig.update_xaxes(title_text="Date")
        fig.update_yaxes(title_text=yTitle)
        fig.update_layout(showlegend=False, height=1000, title=gTitle, xaxis_rangeslider_visible=True)
        fig.show()

In [ ]:
class stockMarketVisualization:
    def __init__(vl, obj):
        vl.stockList = obj.stockList
        vl.data = obj.data
        vl.compList = obj.compList
    def getGraph(vl):
        fig = sp(shared_xaxes=True, subplot_titles=("Stock Market"), vertical_spacing = .5)
        yTitle = "Price ($)"
        gTitle = "Price"
        for i in vl.data:
            if i != 'Date':
                fig.add_trace(g.Scatter(x=pd.to_datetime(vl.data.Date, infer_datetime_format=True), y=vl.data[i].astype("float"), name = i))
        fig.update_xaxes(title_text="Date")
        fig.update_yaxes(title_text=yTitle)
        fig.update_layout(showlegend=False, height=1000, title=gTitle, xaxis_rangeslider_visible=True)
        fig.show()
    def summaryGr(vl):
        plt.figure(figsize=(15, 10))
        for i, company in enumerate(vl.compList, 1):
            plt.subplot(2, 2, i)
            company['Adj Close'].plot()
            plt.ylabel('Adjusted Close')
            plt.xlabel(None)
            plt.title(f"{vl.stockList[i - 1][1]}", fontsize=24)
        plt.tight_layout()

In [ ]:
stock = StockMarket(stList)

In [ ]:
stock.getStockData()
stock.setToCsv()
stock.getReadAndCleanCsv()

In [ ]:
graph = stockMarketVisualization(stock)

In [ ]:
graph.summaryGr()

In [ ]:
graph.getGraph()

In [ ]:
data = stockMarketCrawling(stList)